# U-Net Model for RV Segmentation

## Training U-Net CNN

In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib as mplt
%matplotlib inline

from src import data,unet

Using TensorFlow backend.


In [2]:
from keras import utils

train_dir = "/home/ubuntu/training/TrainingSet"


images=[]
inner_masks=[]
outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    images += imgdata.labeled_images
    inner_masks += imgdata.endo_masks.values()
    outer_masks += imgdata.epi_masks.values()

images = np.asarray(images)[:,:,:,None].astype('float64')
i_masks = np.asarray(inner_masks)
o_masks = np.asarray(outer_masks)

dims = i_masks.shape
classes = len(set(i_masks[0].flatten()))
new_shape = dims + (classes,)
i_masks = utils.to_categorical(i_masks).reshape(new_shape)
o_masks = utils.to_categorical(o_masks).reshape(new_shape)

print("There are %d total training images." % len(images))
print("There are %d total inner masks." % len(inner_masks))
print("There are %d total outer masks." % len(outer_masks))


There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [3]:
unet_conv = unet.UNet()

model = unet_conv.get_unet(height=216,width=256,channels=1,features=32,steps=3,padding='same')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 216, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 216, 256, 32)  320         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 216, 256, 32)  9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 108, 128, 32)  0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [4]:
from keras.optimizers import Adam
from keras import backend as K

def dice_coef(y_true, y_pred):
        flat_y_true = K.flatten(y_true)
        flat_y_preds = K.flatten(y_pred)
        intersection = K.sum(flat_y_true*flat_y_preds)
        return (2. * intersection + 1.) / (K.sum(flat_y_true)+K.sum(flat_y_preds))


model.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy',metrics=[dice_coef])

In [7]:
#TODO: 1. split data into training and validation set
#      2. Augment the data
#      3. Train model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from math import ceil

seed = 0

validation_split=0.2

epochs = 20 
batch_size = 20


split_index = int((1 - validation_split) * len(images))

train_steps = ceil(split_index / batch_size)
val_steps = ceil((len(images)-split_index )/batch_size)

train_images = images[:split_index]
train_inner_masks = i_masks[:split_index]

validation_images = images[split_index:]
validation_inner_masks = i_masks[split_index:]


data_gen_args = dict(rotation_range=180,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.01,
                     fill_mode='nearest')

train_images_datagen = ImageDataGenerator(**data_gen_args)
train_masks_datagen = ImageDataGenerator(**data_gen_args)


seed = 1
#height,width,channels = train_images[0].shape
#print(train_images[0].shape)
#print(train_inner_masks[0].shape)
train_images_datagen.fit(train_images,augment=True,seed=seed)
train_masks_datagen.fit(train_inner_masks,augment=True,seed=seed)

train_images_generator = train_images_datagen.flow(train_images, y=None, seed=seed)
train_masks_generator = train_images_datagen.flow(train_inner_masks, y=None, seed=seed)


train_generator = zip(train_images_generator, train_masks_generator)


/usr/local/lib/python3.5/dist-packages/Keras-2.0.8-py3.5.egg/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
/usr/local/lib/python3.5/dist-packages/Keras-2.0.8-py3.5.egg/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).


In [8]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.hdf5', verbose=1, save_best_only=True)

model.fit_generator(train_generator,steps_per_epoch=train_steps,epochs=epochs,verbose=1,callbacks=[checkpointer],
                   validation_data=(validation_images,validation_inner_masks),validation_steps=val_steps)

Epoch 1/20
10/10 [==============================] - 18s - loss: 0.7289 - dice_coef: 0.4828 - val_loss: 0.7136 - val_dice_coef: 0.4900
Epoch 2/20
10/10 [==============================] - 16s - loss: 0.7115 - dice_coef: 0.4910 - val_loss: 0.7045 - val_dice_coef: 0.4944
Epoch 3/20
10/10 [==============================] - 15s - loss: 0.7041 - dice_coef: 0.4946 - val_loss: 0.6997 - val_dice_coef: 0.4967
Epoch 4/20
10/10 [==============================] - 17s - loss: 0.6993 - dice_coef: 0.4970 - val_loss: 0.6969 - val_dice_coef: 0.4981
Epoch 5/20
10/10 [==============================] - 15s - loss: 0.6970 - dice_coef: 0.4981 - val_loss: 0.6950 - val_dice_coef: 0.4991
Epoch 6/20
10/10 [==============================] - 17s - loss: 0.6951 - dice_coef: 0.4990 - val_loss: 0.6935 - val_dice_coef: 0.4999
Epoch 7/20
10/10 [==============================] - 15s - loss: 0.6935 - dice_coef: 0.4998 - val_loss: 0.6922 - val_dice_coef: 0.5005
Epoch 8/20
10/10 [==============================] - 17s - loss